In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    count = 0
    while True:
        count += 1

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        with open(f'depthmaps/depth_map_{count}.npy', 'wb') as f:
            np.save(f, depth_image)
        color_image = np.asanyarray(color_frame.get_data())
        color_image.flags.writeable = False
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)        
        results = hands.process(color_image)
        color_image.flags.writeable = True
        color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)
        if results.multi_handedness:
            handedness = [handedness.classification[0].label for handedness in results.multi_handedness]
#         print(handedness)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                hand_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x*640)
                hand_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y*480)
                print(hand_x, hand_y)
                color_image = cv2.circle(color_image, (hand_x, hand_y), 10, (255, 0, 0), 2)
#                 mp_drawing.draw_landmarks(color_image,
#                                           hand_landmarks,
#                                           mp_hands.HAND_CONNECTIONS,
#                                           mp_drawing_styles.get_default_hand_landmarks_style(),
#                                           mp_drawing_styles.get_default_hand_connections_style())
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)

finally:

    # Stop streaming
    pipeline.stop()

241 252
233 305
243 300
251 287
284 278
303 266
297 247
294 235
290 224
297 213
303 212
305 220
308 226
305 234
298 238
291 244
275 248
272 244
262 241
248 238
248 233
243 229
233 230
233 224
225 230
223 234
220 238
215 240
217 243
215 246
216 245
215 241
215 236
214 235
217 232
216 230
219 227
223 224
219 224
219 224
222 218
223 214
226 206
221 201
208 198
203 194
202 185
199 178
196 170
194 164
194 161
191 158
180 158
170 176
189 161
178 158
168 156
181 157
170 157
172 156
185 153
163 153
164 151
170 149
183 140
169 145
175 139
176 132
182 133
175 129
179 125
167 135
172 129
163 137
174 129
165 135
180 128
164 131
163 138
167 131
176 131
174 132
180 127
178 123
180 124
179 122
177 117
178 115
176 116
178 113
177 113
175 115
175 110
173 111
172 109
169 102
167 106
163 104
160 100
162 100
150 100
147 89
147 96
140 86
142 84
135 78
138 80
128 70
125 69
114 59
109 53
103 55
96 53
99 85
90 55
101 76
87 51
99 63
78 51
94 60
72 52
72 56
63 56
47 51
50 59
42 61
42 64
38 58
34 63
34 56
33 56


KeyboardInterrupt: 